Python project for Data Engineer - ETL
by Valerija Ravdo

In [42]:
import glob
import pandas as pd
from datetime import datetime

In [43]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-08-24 20:23:54--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: 'bank_market_cap_1.json'

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-08-24 20:23:54 (384 MB/s) - 'bank_market_cap_1.json' saved [2815/2815]

--2023-08-24 20:23:55--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tran

In [44]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [45]:
columns=['Name','Market Cap (US$ Billion)']

In [46]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [47]:
columns=['Name','Market Cap (US$ Billion)']

In [48]:
#EXTRACT FUNCTION:
def extract():
   df=pd.DataFrame(columns=columns)
   for json in glob.glob('*.json'):
      df=df.append(extract_from_json(json), ignore_index=True)
   return df

In [49]:
#EXCHANGE RATE FUNCTION:
exchange=pd.read_csv('exchange_rates.csv', index_col=0)
exchange_rate_series=exchange.loc['GBP']
exchange_rate = exchange_rate_series['Rates']

In [50]:
#TRANSFORM FUNCTION:
def transform(data):
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].astype(float)
    data['Market Cap (US$ Billion)']=round(data['Market Cap (US$ Billion)']*exchange_rate, 3)
    data=data.rename(columns={'Market Cap (US$ Billion)':'Market Cap (GBP$ Billion)'})
    return data

In [51]:
#LOAD FUNCTION
def load(data_to_load):
    targetfile='bank_market_cap_gbp.csv'
    data_to_load.to_csv(targetfile, index=False)

In [52]:
#LOGGING FUNCTION
def log(logging_msg):
    ts_format = '%Y-%h-%d-%H:%M:%S'
    dt_now = datetime.now()
    ts = dt_now.strftime(ts_format)
    with open("log_file.txt","a") as f:
        f.write(ts + ',' + logging_msg + '\n')

In [53]:
#RUNNING THE ETL PROCESS

log("ETL Job Started")

log("Extract phase Started")
df=extract()
print(df.head(5))

                                      Name Market Cap (US$ Billion)
0                           JPMorgan Chase                  390.934
1  Industrial and Commercial Bank of China                  345.214
2                          Bank of America                  325.331
3                              Wells Fargo                  308.013
4                  China Construction Bank                  257.399


/var/folders/8p/r5s07nlj3fq1b7kx_14km9km0000gn/T/ipykernel_37263/4024409645.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(extract_from_json(json), ignore_index=True)
/var/folders/8p/r5s07nlj3fq1b7kx_14km9km0000gn/T/ipykernel_37263/4024409645.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(extract_from_json(json), ignore_index=True)


In [54]:
log("Extract phase Ended")
df

log("Transform phase Started")
data=transform(df)
print(data.head(5))

                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519


In [55]:
log("Transform phase Ended")
data
log("Load phase Started")
load(data)
log("Load phase Ended")
log("ETL Job Ended")